In [3]:
# punkt_tab 문제를 우회하는 소셜미디어 분석 코드

# 패키지 설치
!pip install textblob wordcloud matplotlib seaborn nltk pandas numpy scikit-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from textblob import TextBlob
from wordcloud import WordCloud
import random
import os

# 필요한 NLTK 리소스 명시적으로 다운로드
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

print("NLTK 리소스 다운로드 완료")

# punkt_tab 에러 우회를 위해 word_tokenize 대신 split 사용
def simple_tokenize(text):
    """간단한 토큰화 함수"""
    if not isinstance(text, str):
        return []
    # 소문자 변환
    text = text.lower()
    # URL 제거
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    # 특수문자를 공백으로 대체
    text = re.sub(r'[^\w\s]', ' ', text)
    # 숫자 제거
    text = re.sub(r'\d+', '', text)
    # 여러 공백을 하나로 통합
    text = re.sub(r'\s+', ' ', text).strip()
    # 간단히 공백으로 분리
    return text.split()

# 결과 폴더 생성
if not os.path.exists("reports"):
    os.makedirs("reports")
    print("'reports' 폴더가 생성되었습니다.")

# 가상 소셜미디어 데이터 생성 함수
def generate_mock_social_data(brand_names, n_samples=1000, start_date=None, end_date=None):
    """소셜미디어 가상 데이터 생성"""
    if start_date is None:
        start_date = datetime.now() - timedelta(days=30)
    if end_date is None:
        end_date = datetime.now()

    # 플랫폼 목록
    platforms = ['Twitter', 'Instagram', 'Facebook', 'YouTube', 'TikTok']

    # 감성 표현 패턴 (브랜드별로 다른 감성 패턴을 갖도록 설정)
    positive_patterns = {
        brand: [
            f"I love {brand}!", f"{brand} is amazing", f"Best experience with {brand}",
            f"{brand} products are great", f"Excellent service from {brand}",
            f"{brand}는 최고입니다", f"{brand} 제품이 정말 좋아요", f"역시 {brand}는 믿을 수 있어요"
        ] for brand in brand_names
    }

    negative_patterns = {
        brand: [
            f"Disappointed with {brand}", f"{brand} service is terrible", f"Never buying from {brand} again",
            f"Poor quality {brand} products", f"{brand} needs to improve",
            f"{brand} 제품에 실망했어요", f"{brand} 서비스가 나빠졌어요", f"{brand}는 개선이 필요해요"
        ] for brand in brand_names
    }

    neutral_patterns = {
        brand: [
            f"Just got a new {brand} product", f"Anyone tried {brand}?", f"Thinking about {brand}",
            f"{brand} has new items", f"Seen the {brand} ads?",
            f"{brand} 신제품이 나왔네요", f"{brand} 써보신 분?", f"{brand} 광고 보셨나요?"
        ] for brand in brand_names
    }

    # 데이터 생성
    data = []
    for _ in range(n_samples):
        brand = random.choice(brand_names)
        platform = random.choice(platforms)

        # 날짜 생성
        days_diff = (end_date - start_date).days
        random_days = random.randint(0, days_diff)
        post_date = start_date + timedelta(days=random_days)

        # 감성 선택 (브랜드별로 다른 감성 분포를 갖도록 설정)
        sentiment_dist = {
            brand_names[0]: [0.6, 0.2, 0.2],  # 첫 번째 브랜드는 긍정적
            brand_names[1]: [0.2, 0.6, 0.2],  # 두 번째 브랜드는 부정적
        }

        # 기본 감성 분포
        if brand not in sentiment_dist:
            sentiment_dist[brand] = [0.33, 0.33, 0.34]

        sentiment_type = random.choices(['positive', 'negative', 'neutral'], weights=sentiment_dist[brand])[0]

        # 감성에 따른 패턴 선택
        if sentiment_type == 'positive':
            content = random.choice(positive_patterns[brand])
            sentiment_score = random.uniform(0.5, 1.0)
        elif sentiment_type == 'negative':
            content = random.choice(negative_patterns[brand])
            sentiment_score = random.uniform(-1.0, -0.2)
        else:
            content = random.choice(neutral_patterns[brand])
            sentiment_score = random.uniform(-0.2, 0.5)

        # 댓글 수, 좋아요 수, 공유 수 생성
        if sentiment_type == 'positive':
            comments = random.randint(0, 100)
            likes = random.randint(50, 500)
            shares = random.randint(10, 200)
        elif sentiment_type == 'negative':
            comments = random.randint(0, 200)  # 부정적인 내용이 댓글을 더 많이 받는 경향
            likes = random.randint(0, 100)
            shares = random.randint(0, 50)
        else:
            comments = random.randint(0, 50)
            likes = random.randint(0, 200)
            shares = random.randint(0, 100)

        # 해시태그 생성
        hashtags = [f"#{brand.lower()}", f"#brand", f"#{random.choice(['good', 'new', 'review', 'experience', '추천', '신상', '리뷰'])}"]
        random.shuffle(hashtags)
        hashtags = hashtags[:random.randint(1, 3)]

        # 데이터 항목 추가
        data.append({
            'platform': platform,
            'brand': brand,
            'content': content + " " + " ".join(hashtags),
            'date': post_date,
            'true_sentiment': sentiment_type,
            'sentiment_score': sentiment_score,
            'comments': comments,
            'likes': likes,
            'shares': shares
        })

    return pd.DataFrame(data)

# 언어 감지 함수
def detect_language(text):
    """텍스트의 언어 감지"""
    if not isinstance(text, str) or len(text.strip()) == 0:
        return 'unknown'

    # 한글 패턴
    korean_pattern = re.compile(r'[ㄱ-ㅎㅏ-ㅣ가-힣]')

    # 영어 패턴
    english_pattern = re.compile(r'[a-zA-Z]')

    # 한글 문자 수
    korean_chars = len(korean_pattern.findall(text))

    # 영어 문자 수
    english_chars = len(english_pattern.findall(text))

    # 어떤 언어가 더 많은지 판단
    if korean_chars > english_chars:
        return 'korean'
    elif english_chars > 0:
        return 'english'
    else:
        return 'unknown'

# 텍스트 전처리 함수 (punkt_tab 에러 우회)
def preprocess_text(text, lang='english'):
    """텍스트 전처리 함수"""
    if pd.isna(text) or not isinstance(text, str) or len(text.strip()) == 0:
        return ""

    # 소문자 변환
    text = text.lower()

    # URL 제거
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)

    # 특수문자 제거
    text = re.sub(r'[^\w\s]', '', text)

    # 숫자 제거
    text = re.sub(r'\d+', '', text)

    # 토큰화 - 간단한 split 사용
    tokens = simple_tokenize(text)

    # 불용어 제거
    if lang == 'english':
        try:
            stop_words = set(stopwords.words('english'))
            tokens = [word for word in tokens if word not in stop_words and len(word) > 1]
        except:
            # 불용어 처리 실패시 빈 집합으로 처리
            tokens = [word for word in tokens if len(word) > 1]

    # 토큰 합치기
    processed_text = ' '.join(tokens)

    return processed_text

# 감성 분석 함수
def analyze_sentiment(text, lang='english'):
    """텍스트 감성 분석"""
    if pd.isna(text) or not isinstance(text, str) or len(text.strip()) == 0:
        return 0.0

    # 영어 텍스트 감성 분석
    if lang == 'english':
        try:
            analysis = TextBlob(text)
            return analysis.sentiment.polarity
        except:
            return 0.0

    # 한국어 텍스트 감성 분석 (간단한 키워드 기반)
    elif lang == 'korean':
        positive_words = ['좋은', '최고', '훌륭', '만족', '추천', '좋아', '행복', '긍정']
        negative_words = ['나쁜', '최악', '실망', '불만', '안좋', '싫어', '불행', '부정']

        score = 0
        for word in positive_words:
            if word in text:
                score += 0.2
        for word in negative_words:
            if word in text:
                score -= 0.2

        return max(-1.0, min(1.0, score))  # -1.0 ~ 1.0 범위로 제한

    return 0.0

# 토픽 모델링 함수
def extract_topics(df, text_column, n_topics=5, n_features=1000, n_top_words=10):
    """텍스트에서 토픽 추출"""
    # 결측치 제거 및 정제
    texts = df[text_column].fillna('').tolist()
    texts = [text for text in texts if isinstance(text, str) and len(text.strip()) > 0]

    if len(texts) == 0:
        return {'topics': [], 'topic_weights': []}

    # TF-IDF 벡터화
    try:
        tfidf_vectorizer = TfidfVectorizer(
            max_features=n_features,
            stop_words='english',
            max_df=0.95,
            min_df=2
        )
        tfidf = tfidf_vectorizer.fit_transform(texts)

        # LDA 모델 학습
        lda = LatentDirichletAllocation(
            n_components=n_topics,
            max_iter=10,
            learning_method='online',
            random_state=42
        )
        lda.fit(tfidf)

        # 단어 특성 이름
        feature_names = tfidf_vectorizer.get_feature_names_out()

        # 토픽별 주요 단어 추출
        topics = []
        for topic_idx, topic in enumerate(lda.components_):
            top_features_idx = topic.argsort()[:-n_top_words - 1:-1]
            top_features = [feature_names[i] for i in top_features_idx]
            topics.append({
                'id': topic_idx,
                'words': top_features,
                'weights': topic[top_features_idx].tolist()
            })

        # 문서별 토픽 가중치
        doc_topic_weights = lda.transform(tfidf)

        return {
            'topics': topics,
            'topic_weights': doc_topic_weights
        }
    except Exception as e:
        print(f"토픽 모델링 에러: {str(e)}")
        # 에러 발생시 빈 결과 반환
        return {
            'topics': [{'id': i, 'words': [f'topic_{i}'], 'weights': [0.0]} for i in range(n_topics)],
            'topic_weights': np.zeros((len(texts), n_topics))
        }

# 시각화 함수들
def plot_sentiment_trend(df, brand_column='brand', date_column='date', sentiment_column='sentiment_score'):
    """시간에 따른 브랜드 감성 변화 시각화"""
    # 데이터 준비
    df_copy = df.copy()
    df_copy['date_day'] = pd.to_datetime(df_copy[date_column]).dt.date

    # 일별 평균 감성 점수
    daily_sentiment = df_copy.groupby([brand_column, 'date_day'])[sentiment_column].mean().reset_index()

    # 그래프 그리기
    fig, ax = plt.subplots(figsize=(12, 6))

    for brand, group in daily_sentiment.groupby(brand_column):
        ax.plot(group['date_day'], group[sentiment_column], marker='o', linestyle='-', label=brand)

    ax.axhline(y=0, color='r', linestyle='--', alpha=0.3)
    ax.set_title('브랜드별 일일 평균 감성 점수 변화')
    ax.set_xlabel('날짜')
    ax.set_ylabel('감성 점수')
    ax.legend()
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    return fig

def plot_sentiment_distribution(df, brand_column='brand', sentiment_column='sentiment_score'):
    """브랜드별 감성 분포 시각화"""
    fig, ax = plt.subplots(figsize=(12, 6))

    # 브랜드별로 그룹화하여 히스토그램 그리기
    for brand in df[brand_column].unique():
        brand_data = df[df[brand_column] == brand]
        ax.hist(brand_data[sentiment_column], bins=20, alpha=0.5, label=brand)

    ax.set_title('브랜드별 감성 점수 분포')
    ax.set_xlabel('감성 점수')
    ax.set_ylabel('빈도')
    ax.legend()
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    return fig

def plot_engagement_metrics(df, brand_column='brand', metrics=['comments', 'likes', 'shares']):
    """브랜드별 참여 지표 시각화"""
    # 브랜드별 평균 지표 계산
    engagement_data = df.groupby(brand_column)[metrics].mean().reset_index()

    # 차트 준비
    fig, ax = plt.subplots(figsize=(12, 6))

    # 막대 위치
    x = np.arange(len(engagement_data[brand_column]))
    width = 0.25

    # 막대 그래프 그리기
    for i, metric in enumerate(metrics):
        offset = width * (i - len(metrics)/2 + 0.5)
        ax.bar(x + offset, engagement_data[metric], width, label=metric)

    # 차트 설정
    ax.set_title('브랜드별 평균 참여 지표')
    ax.set_xlabel('브랜드')
    ax.set_ylabel('평균 참여 수')
    ax.set_xticks(x)
    ax.set_xticklabels(engagement_data[brand_column])
    ax.legend()
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    return fig

def plot_platform_sentiment(df, brand_column='brand', platform_column='platform', sentiment_column='sentiment_score'):
    """플랫폼별 브랜드 감성 비교"""
    # 플랫폼별, 브랜드별 평균 감성 점수 계산
    platform_sentiment = df.groupby([platform_column, brand_column])[sentiment_column].mean().reset_index()

    # 히트맵 데이터 준비
    pivot_table = platform_sentiment.pivot(index=platform_column, columns=brand_column, values=sentiment_column)

    # NaN 값 처리
    pivot_table = pivot_table.fillna(0)

    # 히트맵 그리기
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.heatmap(pivot_table, annot=True, cmap='RdBu_r', center=0, vmin=-1, vmax=1, ax=ax)
    ax.set_title('플랫폼별 브랜드 평균 감성 점수')

    plt.tight_layout()
    return fig

def generate_wordcloud(df, brand, text_column='processed_content'):
    """브랜드별 워드클라우드 생성"""
    # 브랜드 데이터 필터링
    brand_data = df[df['brand'] == brand]

    # 모든 텍스트 연결
    text = ' '.join(brand_data[text_column].fillna(''))

    if not text.strip():
        # 텍스트가 비어있으면 기본 텍스트 제공
        text = f"{brand} brand analysis"

    # 워드클라우드 생성
    wordcloud = WordCloud(
        width=800,
        height=400,
        background_color='white',
        max_words=100,
        contour_width=3,
        contour_color='steelblue'
    ).generate(text)

    # 시각화
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.axis('off')
    ax.set_title(f'{brand} 브랜드 워드클라우드')

    plt.tight_layout()
    return fig

# 종합 보고서 생성 함수
def generate_brand_report(df, brand, output_dir=None):
    """브랜드 평판 분석 보고서 생성"""
    # 브랜드 데이터 필터링
    brand_data = df[df['brand'] == brand].copy()

    if len(brand_data) == 0:
        print(f"Error: No data available for brand '{brand}'")
        return {}

    # 기본 정보
    total_mentions = len(brand_data)
    time_period = f"{brand_data['date'].min().date()} to {brand_data['date'].max().date()}"

    # 감성 분석 요약
    avg_sentiment = brand_data['sentiment_score'].mean()
    sentiment_std = brand_data['sentiment_score'].std()
    positive_pct = (brand_data['sentiment_score'] > 0.3).mean() * 100
    negative_pct = (brand_data['sentiment_score'] < -0.3).mean() * 100
    neutral_pct = ((brand_data['sentiment_score'] >= -0.3) & (brand_data['sentiment_score'] <= 0.3)).mean() * 100

    # 참여 지표 요약
    avg_comments = brand_data['comments'].mean()
    avg_likes = brand_data['likes'].mean()
    avg_shares = brand_data['shares'].mean()

    # 플랫폼 분석
    platform_distribution = brand_data['platform'].value_counts().to_dict()
    platform_sentiment = brand_data.groupby('platform')['sentiment_score'].mean().to_dict()

    # 시간적 트렌드
    brand_data['date_day'] = pd.to_datetime(brand_data['date']).dt.date
    daily_sentiment = brand_data.groupby('date_day')['sentiment_score'].mean().to_dict()

    # 결과 통합
    report = {
        'brand': brand,
        'analysis_period': time_period,
        'total_mentions': total_mentions,
        'sentiment': {
            'average': avg_sentiment,
            'std_deviation': sentiment_std,
            'positive_percentage': positive_pct,
            'neutral_percentage': neutral_pct,
            'negative_percentage': negative_pct
        },
        'engagement': {
            'avg_comments': avg_comments,
            'avg_likes': avg_likes,
            'avg_shares': avg_shares
        },
        'platforms': {
            'distribution': platform_distribution,
            'sentiment': platform_sentiment
        },
        'daily_sentiment': daily_sentiment
    }

    # 시각화 (선택적으로 저장)
    if output_dir:
        # 감성 분포
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.histplot(brand_data['sentiment_score'], kde=True, bins=20, ax=ax)
        ax.set_title(f'{brand} 감성 점수 분포')
        ax.set_xlabel('감성 점수')
        ax.set_ylabel('빈도')
        ax.axvline(x=0, color='r', linestyle='--')
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f"{output_dir}/{brand}_sentiment_distribution.png")
        plt.close()

        # 플랫폼별 분포
        fig, ax = plt.subplots(figsize=(10, 6))
        platform_counts = brand_data['platform'].value_counts()
        ax.pie(platform_counts, labels=platform_counts.index, autopct='%1.1f%%')
        ax.set_title(f'{brand} 플랫폼별 언급 분포')
        plt.tight_layout()
        plt.savefig(f"{output_dir}/{brand}_platform_distribution.png")
        plt.close()

        # 시간적 추이
        fig, ax = plt.subplots(figsize=(12, 6))
        daily_data = brand_data.groupby('date_day')['sentiment_score'].mean().reset_index()
        ax.plot(daily_data['date_day'], daily_data['sentiment_score'], marker='o')
        ax.set_title(f'{brand} 일별 평균 감성 점수')
        ax.set_xlabel('날짜')
        ax.set_ylabel('감성 점수')
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f"{output_dir}/{brand}_sentiment_trend.png")
        plt.close()

        # 워드클라우드
        wc_fig = generate_wordcloud(brand_data, brand, 'content')
        wc_fig.savefig(f"{output_dir}/{brand}_wordcloud.png")
        plt.close(wc_fig)

    return report

# 메인 실행 함수
def main():
    """소셜미디어 브랜드 평판 분석 주요 실행 함수"""
    print("===== 브랜드 평판 분석 시작 =====")

    # 1. 분석할 브랜드 설정
    brands = ['Samsung', 'Apple']

    # 2. 가상 데이터 생성
    print("소셜미디어 데이터 수집 중...")
    start_date = datetime.now() - timedelta(days=30)
    df = generate_mock_social_data(brands, n_samples=2000, start_date=start_date)
    print(f"총 {len(df)}개의 데이터를 수집했습니다.")

    # 3. 데이터 전처리
    print("데이터 전처리 중...")
    # 언어 감지
    df['language'] = df['content'].apply(detect_language)
    print(f"언어 감지 완료: {df['language'].value_counts().to_dict()}")

    # 텍스트 전처리
    df['processed_content'] = df.apply(lambda row: preprocess_text(row['content'], row['language']), axis=1)
    print("텍스트 전처리 완료")

    # 4. 감성 분석
    print("감성 분석 중...")
    # 이미 데이터에 감성 점수가 있으므로 그대로 사용
    df['analyzed_sentiment'] = df['sentiment_score']
    print(f"감성 분석 완료: 평균 감성 점수 = {df['analyzed_sentiment'].mean():.2f}")

    # 5. 토픽 모델링
    print("토픽 모델링 중...")
    topics_result = extract_topics(df, 'processed_content', n_topics=5)

    if 'topics' in topics_result and topics_result['topics']:
        print(f"토픽 모델링 완료: {len(topics_result['topics'])}개의 토픽 추출")
        # 주요 토픽 출력
        for i, topic in enumerate(topics_result['topics']):
            if 'words' in topic and topic['words']:
                print(f"토픽 {i+1}: {', '.join(topic['words'][:5])}")
    else:
        print("토픽 모델링 결과가 없습니다.")

    # 6. 결과 시각화
    print("결과 시각화 중...")

    try:
        # 전체 브랜드 감성 트렌드
        sentiment_trend_fig = plot_sentiment_trend(df)
        sentiment_trend_fig.savefig("reports/brand_sentiment_trend.png")
        plt.close(sentiment_trend_fig)
        print("브랜드 감성 트렌드 시각화 완료")

        # 브랜드별 감성 분포
        sentiment_dist_fig = plot_sentiment_distribution(df)
        sentiment_dist_fig.savefig("reports/brand_sentiment_distribution.png")
        plt.close(sentiment_dist_fig)
        print("브랜드별 감성 분포 시각화 완료")

        # 브랜드별 참여 지표
        engagement_fig = plot_engagement_metrics(df)
        engagement_fig.savefig("reports/brand_engagement_metrics.png")
        plt.close(engagement_fig)
        print("브랜드별 참여 지표 시각화 완료")

        # 플랫폼별 브랜드 감성
        platform_fig = plot_platform_sentiment(df)
        platform_fig.savefig("reports/platform_brand_sentiment.png")
        plt.close(platform_fig)
        print("플랫폼별 브랜드 감성 시각화 완료")
    except Exception as e:
        print(f"시각화 중 오류 발생: {str(e)}")

    # 7. 브랜드별 종합 보고서 생성
    print("브랜드 보고서 생성 중...")
    reports = {}
    for brand in brands:
        print(f"{brand} 브랜드 분석 중...")
        try:
            report = generate_brand_report(df, brand, output_dir="reports")
            reports[brand] = report

            # 보고서 요약 출력
            print(f"\n{brand} 브랜드 평판 분석 요약:")
            print(f"총 언급 수: {report['total_mentions']}")
            print(f"평균 감성 점수: {report['sentiment']['average']:.2f}")
            print(f"긍정 언급 비율: {report['sentiment']['positive_percentage']:.1f}%")
            print(f"부정 언급 비율: {report['sentiment']['negative_percentage']:.1f}%")

            # 가장 많이 언급된 플랫폼
            if report['platforms']['distribution']:
                most_platform = max(report['platforms']['distribution'], key=report['platforms']['distribution'].get)
                print(f"가장 많이 언급된 플랫폼: {most_platform}")
        except Exception as e:
            print(f"{brand} 브랜드 분석 중 오류 발생: {str(e)}")

    print("\n===== 분석이 완료되었습니다. =====")
    print(f"결과 이미지는 'reports' 폴더에 저장되었습니다.")

    # 데이터프레임 미리보기 출력
    print("\n데이터 미리보기:")
    print(df[['brand', 'platform', 'content', 'sentiment_score', 'comments', 'likes', 'shares']].head())

    return df, reports, topics_result

# 코드 실행
if __name__ == "__main__":
    try:
        df, reports, topics = main()
        print("프로그램이 성공적으로 실행되었습니다.")
    except Exception as e:
        print(f"프로그램 실행 중 오류가 발생했습니다: {str(e)}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


NLTK 리소스 다운로드 완료
===== 브랜드 평판 분석 시작 =====
소셜미디어 데이터 수집 중...
총 2000개의 데이터를 수집했습니다.
데이터 전처리 중...
언어 감지 완료: {'english': 1955, 'korean': 45}
텍스트 전처리 완료
감성 분석 중...
감성 분석 완료: 평균 감성 점수 = 0.09
토픽 모델링 중...
토픽 모델링 완료: 5개의 토픽 추출
토픽 1: apple, samsung, brand, disappointed, buying
토픽 2: amazing, 최고입니다, samsung, apple, apple는
토픽 3: service, apple, samsung, excellent, terrible
토픽 4: samsung, love, experience, best, brand
토픽 5: needs, improve, apple, samsung, 정말
결과 시각화 중...


<ipython-input-3-441b53602161>:329: UserWarning: Glyph 45216 (\N{HANGUL SYLLABLE NAL}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:329: UserWarning: Glyph 51676 (\N{HANGUL SYLLABLE JJA}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:329: UserWarning: Glyph 44048 (\N{HANGUL SYLLABLE GAM}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:329: UserWarning: Glyph 49457 (\N{HANGUL SYLLABLE SEONG}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:329: UserWarning: Glyph 51216 (\N{HANGUL SYLLABLE JEOM}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:329: UserWarning: Glyph 49688 (\N{HANGUL SYLLABLE SU}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:329: UserWarning: Glyph 48652 (\N{HANGUL SYLLABLE BEU}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipy

브랜드 감성 트렌드 시각화 완료


<ipython-input-3-441b53602161>:347: UserWarning: Glyph 44048 (\N{HANGUL SYLLABLE GAM}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:347: UserWarning: Glyph 49457 (\N{HANGUL SYLLABLE SEONG}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:347: UserWarning: Glyph 51216 (\N{HANGUL SYLLABLE JEOM}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:347: UserWarning: Glyph 49688 (\N{HANGUL SYLLABLE SU}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:347: UserWarning: Glyph 48712 (\N{HANGUL SYLLABLE BIN}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:347: UserWarning: Glyph 46020 (\N{HANGUL SYLLABLE DO}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:347: UserWarning: Glyph 48652 (\N{HANGUL SYLLABLE BEU}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipyt

...

<ipython-input-3-441b53602161>:497: UserWarning: Glyph 44048 (\N{HANGUL SYLLABLE GAM}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:497: UserWarning: Glyph 49457 (\N{HANGUL SYLLABLE SEONG}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:497: UserWarning: Glyph 51216 (\N{HANGUL SYLLABLE JEOM}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:497: UserWarning: Glyph 49688 (\N{HANGUL SYLLABLE SU}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:497: UserWarning: Glyph 48712 (\N{HANGUL SYLLABLE BIN}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:497: UserWarning: Glyph 46020 (\N{HANGUL SYLLABLE DO}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:497: UserWarning: Glyph 48516 (\N{HANGUL SYLLABLE BUN}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipyt


Samsung 브랜드 평판 분석 요약:
총 언급 수: 1015
평균 감성 점수: 0.36
긍정 언급 비율: 66.3%
부정 언급 비율: 17.4%
가장 많이 언급된 플랫폼: Facebook
Apple 브랜드 분석 중...


<ipython-input-3-441b53602161>:497: UserWarning: Glyph 44048 (\N{HANGUL SYLLABLE GAM}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:497: UserWarning: Glyph 49457 (\N{HANGUL SYLLABLE SEONG}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:497: UserWarning: Glyph 51216 (\N{HANGUL SYLLABLE JEOM}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:497: UserWarning: Glyph 49688 (\N{HANGUL SYLLABLE SU}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:497: UserWarning: Glyph 48712 (\N{HANGUL SYLLABLE BIN}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:497: UserWarning: Glyph 46020 (\N{HANGUL SYLLABLE DO}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:497: UserWarning: Glyph 48516 (\N{HANGUL SYLLABLE BUN}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipyt


Apple 브랜드 평판 분석 요약:
총 언급 수: 985
평균 감성 점수: -0.19
긍정 언급 비율: 24.6%
부정 언급 비율: 52.6%
가장 많이 언급된 플랫폼: YouTube

===== 분석이 완료되었습니다. =====
결과 이미지는 'reports' 폴더에 저장되었습니다.

데이터 미리보기:
     brand   platform                                   content  \
0  Samsung     TikTok            Seen the Samsung ads? #samsung   
1    Apple     TikTok            Disappointed with Apple #apple   
2    Apple  Instagram  Never buying from Apple again #리뷰 #brand   
3    Apple    Twitter                Apple 제품에 실망했어요 #brand #추천   
4    Apple    Twitter                    Apple는 개선이 필요해요 #apple   

   sentiment_score  comments  likes  shares  
0         0.061480        41    153      80  
1        -0.882304        61      0       6  
2        -0.576144        22     78      20  
3        -0.614167       123     97      21  
4        -0.341904        94     62      36  
프로그램이 성공적으로 실행되었습니다.


<ipython-input-3-441b53602161>:426: UserWarning: Glyph 48652 (\N{HANGUL SYLLABLE BEU}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:426: UserWarning: Glyph 47004 (\N{HANGUL SYLLABLE RAEN}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:426: UserWarning: Glyph 46300 (\N{HANGUL SYLLABLE DEU}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:426: UserWarning: Glyph 50892 (\N{HANGUL SYLLABLE WEO}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:426: UserWarning: Glyph 53364 (\N{HANGUL SYLLABLE KEUL}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:426: UserWarning: Glyph 46972 (\N{HANGUL SYLLABLE RA}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-3-441b53602161>:426: UserWarning: Glyph 50864 (\N{HANGUL SYLLABLE U}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipytho